In [1]:
import numpy as np
import pandas as pd
import torch
import random
from torchvision import datasets, transforms
from torch.utils.data import TensorDataset
from sklearn.preprocessing import MinMaxScaler
import sklearn
from sklearn.preprocessing import OneHotEncoder
from prettytable import PrettyTable
import matplotlib.pyplot as plt
import sys
#from umap.parametric_umap import ParametricUMAP
device = torch.device('cuda:0')
sys.path.append('../../')
from src.models.frequency.sketch_algorithms import SketchQueryalgorithms
from src.scripts.frequency_membership.train_sqn import define_networks
from src.datasets.zipf import get_zipf_probs, get_dataset_samples, ZipfOnlineDatasetLoader
from src.datasets.read_data import load_data

start importing
start importing
done importing!


This notebook evaluates the networks trained with the zipf dataset in the train_sqn.py computing the error with other datasets. This notebook shows a table with the BF/SQNet/RFQNet errors using the different datasets

In [2]:
# Defining paths
hparams = {
    'names_datasets':['MNIST','fashion_MNIST','humans','dogs','KDD','heart','QuickDraw10','adults','bank'], 
    'folder_datasets': '../../results/datasets/datasets_sketch/',
}

# Read config sqnet
import hjson
with open('../../configs/config_membershipmodel_sk100.txt') as json_file:
    hparams_1 = hjson.load(json_file)
    
# Join the two json in hparams
hparams.update(hparams_1)
hparams['folder_datasets'] = '../../results/datasets/datasets_sketch/'
class_read_data = load_data()


In [3]:
# Define parameters
name = 'Min'#'Max'
num = 1
hparams['num_hidden'] = num
hparams["residual_sketch_net"]=1
activation = 'gelu' #'cos'
#hparams['both_sq_rfq']= 1

In [4]:
# Define path networks

# SQNet
hparams['sq_snet']='../../results/networks/networks_sketch_100/networks_membership_sqnet_residual/' + 'snet_sq_'+ name +'.pth' #_lr0.0001
hparams['sq_qnet']='../../results/networks/networks_sketch_100/networks_membership_sqnet_residual/' + 'qnet_sq_'+ name +'.pth'


# RFQnet
hparams['rf_snet']='../../results/networks/networks_sketch_100/networks_membership_rf/' + 'snet_sq_'+ activation +'0.6.pth' #_lr0.0001
hparams['rf_qnet']='../../results/networks/networks_sketch_100/networks_membership_rf/' + 'qnet_sq_'+ activation +'0.6.pth'


hparams['type_lastlayer_sq']= name
hparams['activation'] = activation



In [5]:
# Define networks
hparams['train_SN_QN']=0

SN_rf,QN_rf = define_networks(hparams)
# Load the networks
SN_rf.load_state_dict(torch.load(hparams['rf_snet']))
QN_rf.load_state_dict(torch.load(hparams['rf_qnet']))
SN_rf.eval()
QN_rf.eval()
hparams['train_SN_QN']=1
# Define networks
SN,QN = define_networks(hparams)
# Load the networks 
SN.load_state_dict(torch.load(hparams['sq_snet']))
QN.load_state_dict(torch.load(hparams['sq_qnet']))
SN.eval()
QN.eval()

#compute_test(SN,QN,hparams,device,SN_rf,QN_rf)

QueryNetworkTask(
  (net): Sequential(
    (0): Linear(in_features=100, out_features=2000, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(2000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ResidualFC(
      (layers): Sequential(
        (0): Linear(in_features=2000, out_features=2000, bias=True)
        (1): ReLU()
        (2): BatchNorm1d(2000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (4): ResidualFC(
      (layers): Sequential(
        (0): Linear(in_features=2000, out_features=2000, bias=True)
        (1): ReLU()
        (2): BatchNorm1d(2000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (5): ResidualFC(
      (layers): Sequential(
        (0): Linear(in_features=2000, out_features=2000, bias=True)
        (1): ReLU()
        (2): BatchNorm1d(2000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (6): Linear(in_features=2000, out_features=1000, 

In [6]:
def bloom_filters(list_w,list_d,n_dims,x,y,metric):
    """
    Compute the Bloom filter of a dataset with batches and different d and w.
    The loss is computed doing the mean of the errors of the batches
    """
    errors_bf = []
    
    for i in range(len(list_w)):
        w = list_w[i]
        d = list_d[i]
        bloom_filters = SketchQueryalgorithms(n_dims, w, d,'BF')
        error_BF = []       
        for batch in range(x.shape[0]):
            x_i = x[batch].cpu()
            y_i = y[batch].cpu()
            # Compute BF
            y_hat, s = bloom_filters.SketchQuery_algorithms(x_i)
            error = torch.mean(metric(torch.clamp(y_hat, 0, 1)-y_i.squeeze()))
            error_BF.append(error.item()) 
         
        errors_bf.append(np.mean(error_BF))
        #print(np.mean(error_BF))
    return errors_bf

In [7]:
def compute_traditional_methods(data_test,y,name_dataset,metric):
    """
    Calls the function bloom_filters that compute the Bloom filter of a dataset with batches and different d and w,
    and returns a list with the errors of BF. Each error of the list is computed with a different d and w.
    This function takes the minimum value of the loss computed with different d and w
    """
    # Optimize parameters
    # IMPLEMENT
    n_dims = 1000 # Input dimention
    sketch_size = 100
    list_w = [1,2,4,5,10,20,25,50,100] 
    list_d = [100,50,25,20,10,5,4,2,1] 
    
    # Execute CM/CS
    err_BF = bloom_filters(list_w,list_d,n_dims,data_test,y,metric)
    
    #print(err_BF)
    
    return np.min(err_BF)


In [8]:
def compute_networks(data,y,name_dataset,hparams,metric):
    """
    Evaluates the sqnet and rfqnet networks 
    """
    ## Compute the errors with the trained networks
    ## Use networks
    with torch.no_grad(): # .unsqueeze(0)
        sketch = SN(data.float().to(device), mean_dim=1,type_lastlayer=hparams['type_lastlayer_sq'],order_norm=hparams['order_norm_sq'])
        y_hat = torch.sigmoid(QN(sketch)).squeeze()
        error_sqnet = torch.mean(metric(y_hat.cpu()-y.squeeze()))
        #print('Error sqnet:{}, dataset {}'.format(error_sqnet.item(),name_dataset))  

    with torch.no_grad():
        sketch = SN_rf(data.float().to(device), mean_dim=1,type_lastlayer=hparams['type_lastlayer_rf'],order_norm=hparams['order_norm_rf'])
        y_hat = torch.sigmoid(QN_rf(sketch)).squeeze()
        error_rfqnet = torch.mean(metric(y_hat.cpu()-y.squeeze()))
        #print('Error rfnet:{}, dataset {}'.format(error_rfqnet.item(),name_dataset))  
        
    return error_sqnet.item(),error_rfqnet.item()




In [9]:
def ds_reshape( ds, dim1, dim2 ):
    """
    Reshapes the dimensions of the data to add the batch size dim (Batch_size x num_samples x num_features)
    The input parameters are the data, dim1 (batch_size) and dim2 (num_samples)
    If dim1*dim2 is bigger than the number of rows of the datasets, an error will be printed
    If dim1*dim2 is smaller than the number of rows, data will be trimmed and will have the following dimensions:(Batch_size x num_samples x num_features) 
    If dim1*dim2 is equal to the number of rows, data will have the following dimensions:(Batch_size x num_samples x num_features) 
    """ 
    assert( dim1*dim2 <= ds.shape[0] )
    if  dim1*dim2 < ds.shape[0]:
        print('Warning: data will be trimmed')
        return ds[:dim1*dim2].reshape( (dim1, dim2, ds.shape[1] )) 
    else:
        return ds.reshape( ( dim1, dim2, ds.shape[1] ))

In [15]:
# Define num samples
num_samples = 10

i = 0
x = PrettyTable()
metric = torch.square
x.field_names = ["dataset","BF", "rfqnet", "sqnet"]
list_BF = []
list_sqnet = []
list_rfqnet = []
# For each dataset compute BF/sqnet/rfqnet
for name_dataset in hparams['names_datasets']: 
    print(name_dataset)
    path = hparams['folder_datasets'] + hparams['names_datasets'][i]+'.npz'
    data_train,data_val,data_test = class_read_data.loaddata(path)
  
    # reshape data to have batch, num_samples and features (Bx500x1000)
    if name_dataset == 'dogs':
        data = ds_reshape( data_test,  int(data_test.shape[0]/62), 62 )
    else:
        data = ds_reshape( data_test,  int(data_test.shape[0]/num_samples), num_samples)
        
    y = torch.mean(data.float(), dim=1)
    y[y == 0] = 0
    y[y > 0] = 1    
    print(data.shape[0])
    err_BF=compute_traditional_methods(data,y,name_dataset,metric)
    err_sqnet,err_rf=compute_networks(data,y,name_dataset,hparams,metric)
    i += 1
    x.add_row([name_dataset, round(err_BF,3),round(err_rf,3), round(err_sqnet,3)])
    # Add errors to the list, to later do the mean
    list_BF.append(err_BF)
    list_sqnet.append(err_sqnet)
    list_rfqnet.append(err_rf)

x.add_row(['Average', round(np.mean(list_BF),3), round(np.mean(list_rfqnet),3), round(np.mean(list_sqnet),3)])    
print(x)


MNIST
1200
fashion_MNIST
700
humans
41
dogs
1
KDD
954
heart
3198
QuickDraw10
1000
adults
488
bank
411
+---------------+-------+--------+-------+
|    dataset    |   BF  | rfqnet | sqnet |
+---------------+-------+--------+-------+
|     MNIST     | 0.647 | 0.308  | 0.231 |
| fashion_MNIST | 0.393 | 0.334  | 0.237 |
|     humans    | 0.578 | 0.359  | 0.264 |
|      dogs     | 0.055 | 0.053  | 0.051 |
|      KDD      | 0.218 | 0.195  | 0.177 |
|     heart     | 0.373 | 0.083  | 0.042 |
|  QuickDraw10  | 0.539 |  0.26  | 0.226 |
|     adults    | 0.389 | 0.063  | 0.045 |
|      bank     | 0.519 | 0.092  | 0.068 |
|    Average    | 0.412 | 0.194  | 0.149 |
+---------------+-------+--------+-------+
